In [2]:
from usagrid import s3
import pandas as pd

files = s3.list_files_in_folder("usagrid","landingarea/powerplant")[1:]

dfs = [s3.read_pyarrow_df_from_s3("usagrid",f).to_pandas() for f in files[:]]

df = pd.concat(dfs)

In [33]:
def set_dtypes(df):
    
    data = df.copy()

    dtypes_dict = {'period':"datetime64[ns]",
    'plantCode':"object",
    'plantName':"object",
    'fuel2002':"object",
    'fuelTypeDescription':"object",
    'state':"object",
    'stateDescription':"object",
    'primeMover':"object",
    'average-heat-content':"float64",
    'average-heat-content-units':"object",
    'consumption-for-eg':"float64",
    'consumption-for-eg-units':"object",
    'consumption-for-eg-btu':"float64",
    'generation':"float64",
    'gross-generation':"float64",
    'total-consumption':"float64",
    'total-consumption-units':"object",
    'total-consumption-btu':"float64",
    'consumption-for-eg-btu-units':"object",
    'generation-units':"object",
    'gross-generation-units':"object",
    'total-consumption-btu-units':"object"
    }

    for k, v in dtypes_dict.items():

        data[k] = data[k].astype(v)

    return data

def filter_out_ALL_fuel(df):

    return df.query('fuel2002 != "ALL"')

def filter_out_ALL_prime_movers(df):

    return df.query('primeMover != "ALL"')


def extract_year(df):

    return df.assign(year=df.period.dt.year)

modified = (
df
.pipe(set_dtypes)
.pipe(extract_year)
.pipe(filter_out_ALL_fuel)\
.pipe(filter_out_ALL_prime_movers)
)

In [36]:
modified.groupby("year")[["generation","gross-generation","total-consumption"]].sum()

,generation,gross-generation,total-consumption
year,,,
2022,1.014910e+09,1.067361e+09,3.692905e+09
2023,3.274087e+09,3.432687e+09,1.282050e+10
